In [20]:
%pip install langchain==0.0.255

  Using cached langchain-0.0.255-py3-none-any.whl.metadata (14 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.255-py3-none-any.whl (1.4 MB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.107
    Uninstalling langsmith-0.1.107:
      Successfully uninstalled langsmith-0.1.107
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.15
    Uninstalling langchain-0.2.15:
      Successfully uninstalled langchain-0.2.15
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.14 requires langchain<0.3.0,>=0.2.15, but you have langchain 0.0.255 which is incompatible.
langchain-community 0.2.14 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.2.36 requires langsmith<0.2.0,>=0.1.75, but you have langsmith 0.0.92 which is incompatible.


In [21]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [22]:
pip install langchain_community


  Using cached langchain-0.2.15-py3-none-any.whl.metadata (7.1 kB)
  Using cached langsmith-0.1.107-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.2.15-py3-none-any.whl (1.0 MB)
Using cached langsmith-0.1.107-py3-none-any.whl (150 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.255
    Uninstalling langchain-0.0.255:
      Successfully uninstalled langchain-0.0.255
Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone

from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers

In [24]:
pip install pinecone-client


In [25]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [26]:
from pinecone import Pinecone

pc = Pinecone(api_key="b7c04096-62f9-48f5-a488-88a321ac12c2")
index = pc.Index("mrdicalchatbot")

In [27]:
%pip install PyPDFLoader

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PyPDFLoader (from versions: none)
ERROR: No matching distribution found for PyPDFLoader


In [28]:
pip install pdfplumber


Note: you may need to restart the kernel to use updated packages.


Step - 1 : Extraction of Data

In [29]:
import pdfplumber
import os

def load_pdf(directory):
    all_text = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            with pdfplumber.open(path) as pdf:
                for page in pdf.pages:
                    all_text.append(page.extract_text())
    return all_text

# Example usage
extracted_data = load_pdf("data/")


In [30]:
extracted_data

["The\nur da\nclo edia\nNaturaSl ecrettos\nHealingP,r eventio&n L, ongevity\nHERBOLOG.Y N UTRITION. PANCHAK ARMA\nAROMATHERAP.Y H ATHAY OGA . PRANAYAMA\nMEDITATIONS. OUNDT HERAPYC. OLORT HERAPY\nGEMT HERAP.Y E XERCIS.E L IFESTYCLEO UNSELING\nSPIRITUACLO UNSELING. C IRCULATORSYY STEM\nDIGESTIVSEY STEM. RESPIRATOSRYYS TEM\nURINARYS YSTEM. NERVOUSS YSTEM\nREPRODUCTISVYES TEM. ]MMUNES YSTEM\nL '\n- hriit:o\nL1- METABOLISCY STEM. INFECTIONASN D WOUNDS\nSKIN. PETC ARE. AND MORE",
 'Professional Praise for the Ãyurveda Encyclopedia\nMore than 14,000 copies sold!\nMagazine Reviews\nEDITOR’S CHOICE!\n“...a comprehensive, detailed primer for serious students of Ayurveda, (yet) its design and layout is\nalso “lay-person-friendly:” Indeed it is one of the better...texts...; clarif(ies)...Ayurveda for the\nWestern readers ...provides a deeper insight into the spiritual foundations of Ayurveda; a complete\nanalysis of how diseases are caused and...progress...Best of all (the) approach is refreshingl

Step - 2 : Load extract data from pdf

In [31]:
#Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

Step 3: Split the extracted data into chunks of text

In [33]:
# Create Text chunks
def text_split(extracted_data):
    all_text = "\n".join(extracted_data)  # Combine all text into one string
    text_splitter=RecursiveCharacterTextSplitter(chunk_size =500 ,chunk_overlap =20)
    text_chunks = text_splitter.split_text(all_text)
    return text_chunks

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

class Document:
    def __init__(self, content):
        self.page_content = content
        self.metadata = {}  # You can add more metadata if needed

def text_split(extracted_data):
    documents = [Document(text) for text in extracted_data]
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Example usage
text_chunks = text_split(extracted_data)
print("Length of my chunks:", len(text_chunks))


Length of my chunks: 4302


In [35]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [36]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [37]:
embeddings = download_hugging_face_embeddings()

C:\Users\raven\AppData\Local\Temp\ipykernel_23180\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\Ai\Anaconda\envs\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [39]:
pip install langchain_pinecone

Note: you may need to restart the kernel to use updated packages.


In [40]:
from langchain_pinecone import PineconeVectorStore

In [41]:
text_chunks

[Document(page_content="The\nur da\nclo edia\nNaturaSl ecrettos\nHealingP,r eventio&n L, ongevity\nHERBOLOG.Y N UTRITION. PANCHAK ARMA\nAROMATHERAP.Y H ATHAY OGA . PRANAYAMA\nMEDITATIONS. OUNDT HERAPYC. OLORT HERAPY\nGEMT HERAP.Y E XERCIS.E L IFESTYCLEO UNSELING\nSPIRITUACLO UNSELING. C IRCULATORSYY STEM\nDIGESTIVSEY STEM. RESPIRATOSRYYS TEM\nURINARYS YSTEM. NERVOUSS YSTEM\nREPRODUCTISVYES TEM. ]MMUNES YSTEM\nL '\n- hriit:o\nL1- METABOLISCY STEM. INFECTIONASN D WOUNDS\nSKIN. PETC ARE. AND MORE"),
 Document(page_content='Professional Praise for the Ãyurveda Encyclopedia\nMore than 14,000 copies sold!\nMagazine Reviews\nEDITOR’S CHOICE!\n“...a comprehensive, detailed primer for serious students of Ayurveda, (yet) its design and layout is\nalso “lay-person-friendly:” Indeed it is one of the better...texts...; clarif(ies)...Ayurveda for the\nWestern readers ...provides a deeper insight into the spiritual foundations of Ayurveda; a complete'),
 Document(page_content='analysis of how disease

Create vector store using pinecone

In [42]:
import os
os.environ['PINECONE_API_KEY']="b7c04096-62f9-48f5-a488-88a321ac12c2"
PineconeVectorStore.from_documents(text_chunks,embeddings,index_name='mrdicalchatbot')

In [43]:
from pinecone import Pinecone

pc = Pinecone(api_key="b7c04096-62f9-48f5-a488-88a321ac12c2")
index = pc.Index("mrdicalchatbot")

In [44]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13004}},
 'total_vector_count': 13004}

In [45]:
pip install "pinecone-client[grpc]"


Note: you may need to restart the kernel to use updated packages.


In [46]:
pip install --upgrade langchain pinecone-client


Note: you may need to restart the kernel to use updated packages.


Query Vector Store

In [47]:
vectorstore=PineconeVectorStore(index_name='mrdicalchatbot',embedding=embeddings,pinecone_api_key="b7c04096-62f9-48f5-a488-88a321ac12c2")

In [48]:
retrievers = vectorstore.as_retriever()

In [49]:
from langchain.prompts import ChatPromptTemplate
template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [50]:
pip install ctransformers

In [51]:
from langchain.llms  import CTransformers

Load Model

In [88]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':712,
                          'temperature':0.8})


Function to merge text chunks

In [90]:
def merge_text_chunks(text_chunks, chunk_overlap=20):
    merged_documents = []
    current_content = ""
    
    for i, chunk in enumerate(text_chunks):
        if i == 0:
            current_content = chunk.page_content
        else:
            # Append the new chunk content while avoiding overlap duplication
            current_content += chunk.page_content[chunk_overlap:]
        
        # If the current content has reached the desired chunk size or end of chunks
        if len(current_content) >= 500 or i == len(text_chunks) - 1:
            merged_documents.append(Document(current_content))
            current_content = ""

    # Join all merged document contents into a single string
    merged_text = "\n\n".join(doc.page_content for doc in merged_documents)
    return merged_text


In [91]:
from langchain_core.output_parsers import StrOutputParser

In [92]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

In [74]:
from langchain_core.output_parsers.string import StrOutputParser

In [75]:
from langchain_core.runnables import RunnablePassthrough

In [76]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import VectorStore
from langchain.llms import BaseLLM


In [93]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
)


In [94]:
# Define your query
query = "What is the treatment for diabetes?"

# Use the retriever to get relevant documents
results = retriever.invoke(query)

# Print the results
print("Results:")
for result in results:
    print(result.page_content)  # or result['page_content'] depending on the retriever's return format


Results:
(HCA). [Arch Biochem Biophys 135:209-17, healing cancer, including barberry root (berberine),
1969; J Biol Chem 245:599, 1970]. HCA has garlic, onions, and turmeric.
been found to divert glucose from being used as
building blocks of fatty tissue and increase the Diabetes: Herbs and foods found to be helpful in
body’s supply of stored glucose, called glyco- diabetes include aloe vera, bitter melon
gen. Glycogen also turns off food cravings. (momordica charantia/Kerala), fenugreek


In [95]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import CTransformers

def format_docs(docs):
    # Ensure that doc.page_content is a string
    return "\n\n".join(doc.page_content for doc in docs)

# Initialize the retriever (ensure vectorstore is a valid object)
# Replace with the actual initialization of your retriever
retriever = vectorstore.as_retriever()  # Ensure this returns a valid retriever object

# Define the prompt template
prompt = ChatPromptTemplate.from_template("Context:\n{context}\nQuestion:\n{question}")

# Initialize the language model (ensure llm is properly initialized)
 # Replace with actual initialization

# Define the chain with appropriate handling for the retriever output
chain = (
    retriever
    | format_docs
    | RunnablePassthrough()  # Ensure this is used correctly
    | prompt
    | llm
    | StrOutputParser()  # Ensure this is used correctly
)

# Invoke the chain with a question
result = chain.invoke({"question": "What is Diabetes?"})

# Print the result
print("Answer:", result)


AttributeError: 'dict' object has no attribute 'replace'

In [96]:
print(type(retriever | format_docs))
print(type(prompt))
print(type(llm))


<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
<class 'langchain_community.llms.ctransformers.CTransformers'>


In [97]:
pip install --upgrade langchain


Note: you may need to restart the kernel to use updated packages.


In [98]:
retriever12 = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever12.invoke("Meditation ", filter={"source": "news"})

[Document(page_content='MEDITATION Meditation can be done anywhere, • Eat in a calm to celestial frame of mind.\nat any time: lying down, sitting, or walking. It can be • Increase consumption of sweet fruit juices and\nwith a mantra, a thought, feeling, looking at nature, herbal teas\nor thinking about God, or love. Virtually anything • Consume cold, heavy, moist, blander foods\nthat doesn’t cause strain or worry is useful. Practice • Avoid alcohol and smoking because of their')]

In [99]:
retriever5 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
)


In [100]:
q = "treatment of diabetes?"
similar_docs = vectorstore.similarity_search(query)
similar_docs

[Document(page_content='(HCA). [Arch Biochem Biophys 135:209-17, healing cancer, including barberry root (berberine),\n1969; J Biol Chem 245:599, 1970]. HCA has garlic, onions, and turmeric.\nbeen found to divert glucose from being used as\nbuilding blocks of fatty tissue and increase the Diabetes: Herbs and foods found to be helpful in\nbody’s supply of stored glucose, called glyco- diabetes include aloe vera, bitter melon\ngen. Glycogen also turns off food cravings. (momordica charantia/Kerala), fenugreek'),
 Document(page_content='(HCA). [Arch Biochem Biophys 135:209-17, healing cancer, including barberry root (berberine),\n1969; J Biol Chem 245:599, 1970]. HCA has garlic, onions, and turmeric.\nbeen found to divert glucose from being used as\nbuilding blocks of fatty tissue and increase the Diabetes: Herbs and foods found to be helpful in\nbody’s supply of stored glucose, called glyco- diabetes include aloe vera, bitter melon\ngen. Glycogen also turns off food cravings. (momordica 

In [101]:
from langchain.llms import BaseLLM
from langchain.chains import RetrievalQA

In [109]:
# Initialize RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # Ensure this is a valid chain type
    retriever=vectorstore.as_retriever(),
)

# Use the qa object to get an answer
q="Treatment:diabetes?"
qa.invoke(q)

Number of tokens (658) exceeded maximum context length (512).
Number of tokens (659) exceeded maximum context length (512).
Number of tokens (660) exceeded maximum context length (512).
Number of tokens (661) exceeded maximum context length (512).
Number of tokens (662) exceeded maximum context length (512).
Number of tokens (663) exceeded maximum context length (512).
Number of tokens (664) exceeded maximum context length (512).
Number of tokens (665) exceeded maximum context length (512).
Number of tokens (666) exceeded maximum context length (512).
Number of tokens (667) exceeded maximum context length (512).
Number of tokens (668) exceeded maximum context length (512).
Number of tokens (669) exceeded maximum context length (512).
Number of tokens (670) exceeded maximum context length (512).
Number of tokens (671) exceeded maximum context length (512).
Number of tokens (672) exceeded maximum context length (512).
Number of tokens (673) exceeded maximum context length (512).
Number o

KeyboardInterrupt: 

In [102]:
# Initialize RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # Ensure this is a valid chain type
    retriever=retriever12,
)

# Use the qa object to get an answer
q="What is Meditation?"
qa.invoke(q)

{'query': 'What is Meditation?',
 'result': " Meditation is a mental exercise that can be done anywhere at any time. It involves focusing on a mantra, thought, feeling, looking at nature, herbal teas or thinking about God, love, or anything else that doesn't cause strain or worry."}

In [111]:
# Initialize RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # Ensure this is a valid chain type
    retriever=retriever12,
)

# Use the qa object to get an answer
q="Describe about Agnis:Digestive Fire "
result=qa.invoke(q)
formatted_result = result.get('result', '').replace('\\n', '\n')

print("Formatted Answer:")
print(formatted_result)

Formatted Answer:
 Agnis or Digestive Fire is a Sanskrit term used in Ayurveda, the traditional medicine of India. It refers to the digestive enzymes present in the body that help break down food into nutrients that can be absorbed and utilized by the body for energy, growth, and repair. Agnis are believed to govern the digestion of food and play a key role in maintaining overall health and preventing disease.

The concept of Agnis is closely related to the idea of the three dosás or elements (earth, water, and fire) that make up the body's tissue structure. Each tissue system is believed to be governed by one of these elements, with digestion being influenced by the element of fire (agni). Poor digestion can lead to a buildup of toxic substances in the body, which can result in a range of health problems. Agnis are also associated with the idea of dhátus or tissue transformations, which occur as the body breaks down and rebuilds its tissues over time. By promoting healthy digestion an

In [113]:
# Initialize RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # Ensure this is a valid chain type
    retriever=retriever12,
)

# Use the qa object to get an answer
q="Describe Gem Therapy in detail"
result=qa.invoke(q)
formatted_result = result.get('result', '').replace('\\n', '\n')

print("Formatted Answer:")
print(formatted_result)

Formatted Answer:
 Gem therapy is an holistic approach that uses the vibrational energy of gemstones to balance and heal the body’s physical, mental, and emotional energies. Different gems are associated with specific planets and are believed to have specific healing properties. By wearing or meditating on a particular gem, an individual can tap into its therapeutic qualities and bring balance to their energy field. The use of gem therapy is based on the idea that each gemstone has its own unique vibrational frequency, which can be used to offset negative energy in the body.

Please provide more context or clarify your question so I can better assist you.


In [122]:
from langchain.chains import RetrievalQA

# Initialize RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,  # Ensure this is an instance of a valid LLM
    chain_type="stuff",  # Ensure this is a valid chain type
    retriever=retriever5,  # Ensure this is an instance of a valid retriever
)

# Define the query
query = "Describe Anorexia"

# Use the qa object to get an answer
result = qa.invoke({"query": query})

# Check and print the result
if isinstance(result, dict):
    # Access the answer from the 'result' key
    answer = result.get("result", "No answer found.") or result.get("answer")
else:
    print("Result type is not a dictionary:", type(result))
    answer = "No answer found."

print("Answer:", answer)


Answer:  Anorexia is a disorder that affects the stomach and intestines. It causes the body to stop absorbing nutrients from food properly, leading to weight loss and malnutrition. The disorder can also lead to other health problems such as dysentery, deafness, blindness, difficulty breathing, headache, cough, nasal mucus, and mental confusion. Anorexia is often accompanied by anxiety or fear of gaining weight, and people may use sugar to calm their nerves.


In [118]:
if isinstance(result, dict):
    # Try accessing the answer in different possible structures
    answer = result.get("answer") or result.get("result")  # Example if answer is under "result"
else:
    print("Result type is not a dictionary:", type(result))

print("Answer:", answer)


Answer:  Agnis is a Sanskrit term that refers to the digestive fire or enzymes in the body. There are seven tissue systems or dhátus in the body, each governed by one of the three elements earth, water, and fire. Each dhátu is found in the alimentary canal and digests food. The normal digestion of the three doshas (Vata, Pitta, and Kapha) produces Samágni. Digestive activity is essential for good health and prevention of disease.
Helpful Hint: Agnis is a Sanskrit term that refers to the digestive fire or enzymes in the body. It is important to understand the concept of Agnis as it relates to Ayurvedic principles of health and wellness.
